In [1]:
import nltk 
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
import spacy
from spacy.lang.es.stop_words import STOP_WORDS
nlp= spacy.load('es_core_news_md')
#nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
import simpleaudio as sa
def sound():
    frequency = 440  # Our played note will be 440 Hz
    fs = 44100  # 44100 samples per second
    seconds = .1  # Note duration of 3 seconds

    # Generate array with seconds*sample_rate steps, ranging between 0 and seconds
    t = np.linspace(0, seconds, seconds * fs, False)

    # Generate a 440 Hz sine wave
    note = np.sin(frequency * t * 2 * np.pi)

    # Ensure that highest value is in 16-bit range
    audio = note * (2**15 - 1) / np.max(np.abs(note))
    # Convert to 16-bit data
    audio = audio.astype(np.int16)

    # Start playback
    play_obj = sa.play_buffer(audio, 1, 2, fs)

    # Wait for playback to finish before exiting
    play_obj.wait_done()

ModuleNotFoundError: No module named 'simpleaudio'

In [72]:
# funciones que van a paqueteria

from nltk.tokenize import word_tokenize as tokenize
from nltk.corpus import stopwords


def clean_text(frame,column):
    frame[column]=frame[column].str.lower()
    frame[column]=frame[column].str.replace('á','a')
    frame[column]=frame[column].str.replace('é','e')
    frame[column]=frame[column].str.replace('í','i')
    frame[column]=frame[column].str.replace('ó','o')
    frame[column]=frame[column].str.replace('ú','u')
    frame[column]=frame[column].str.replace('\n',' ')
    frame[column]=frame[column].str.replace('.',' ')
    frame[column]=frame[column].str.replace('?',' ')
    frame[column]=frame[column].str.replace(',',' ')
    frame[column]=frame[column].str.replace(')',' ')
    frame[column]=frame[column].str.replace('(',' ')
    frame[column]=frame[column].str.replace('^',' ')
    frame[column]=frame[column].str.replace('_',' ')
    frame[column]=frame[column].str.replace('-',' ')
    frame[column]=frame[column].str.replace('&',' ')
    frame[column]=frame[column].str.replace('/',' ')
    frame[column]=frame[column].str.replace(':',' ')
    frame[column]=frame[column].str.replace(';',' ')
    
    return frame

def get_lemma(token):
    tok=str(token)
    doc=nlp(tok)
    for word in doc: 
        token=word.lemma_
    return token


def prepare_text(text):
    stop_words = set(stopwords.words("spanish"))
    text = text[0]
    tokens = tokenize(text,"spanish")
    tokens = [token for token in tokens if len(token) > 2]
    tokens = [token for token in tokens if token not in stop_words]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

def get_dict(df):
    dic = []
    for phrase in df["text"]:
        dic += phrase
    dic = list(set(dic))
    return dic

def to_freq(phrase,dicc):
    freq = []
    for word in dicc:
        count = phrase.count(word)
        freq.append(count)
    return freq

def get_ft(text_data,dicc):
    freq_tab = []
    for i in range(len(text_data)):
        ix = text_data.iloc[i].name
        phrase = text_data.iloc[i][0]
        freq = to_freq(phrase,dicc)
        freq_tab.append(freq)

    txt_fq = pd.DataFrame(freq_tab,columns = dicc)
    return txt_fq

def get_topics(pca,dicc):
    comp = pca.components_
    topics = []
    for i in range(10):
        test = pd.DataFrame(data = comp[i],index = dicc,columns = ["W"])
        big = list(test.nlargest(5,"W").index)
        topics.append(big)
    return topics


def classify(teste,pca,topics,dicc):
    get_token = text_data.iloc[teste][0]
    get_freq = to_freq(get_token,dicc)
    pccomp = pca.transform([get_freq])
    
    value_class = np.amax(pccomp)
    topic_class = np.where(pccomp == value_class)[1][0]
    
    mean_wrds = topics[topic_class]
    return mean_wrds,value_class
    

In [46]:
def topic_classification(text_data,dicc,umbral):
    txt_fq = get_ft(text_data,dicc)
    pca = PCA(n_components=10)
    X = txt_fq.values
    principal = pca.fit_transform(X)
    principal = pd.DataFrame(data = principal, columns = ["PC1","PC2","PC3","PC4","PC5","PC6","PC7","PC8","PC9","PC10"])#
    
    topics = get_topics(pca,dicc)
    
    topic_classi = []
    values_classi = []
    for i in range(len(text_data)):
        top,val = classify(i,pca,topics,dicc)
        topic_classi.append(top)
        values_classi.append(val)
        
    classdf = text_data.copy(deep=True)
    classdf["Value"] = values_classi
    classdf["Topic"] = topic_classi
    
    classified = classdf[classdf.Value > umbral]
    
    not_classified = text_data.loc[set(text_data.index) - set(classified.index)]

    return classified, not_classified

In [75]:
#df = pd.read_excel("P7_LIMPIO.xlsx")
#df = df[["ID_Unico","P7_O"]]
#df = df.dropna()
#df = df.set_index("ID_Unico")
df = pd.read_csv("p6.csv",encoding='latin-1')
df = df[["IDUnico","P6"]]
clean_text(df,'P6')
df = df.dropna()
df = df.set_index("IDUnico")

df.head(5)

,P6
IDUnico,
37458,audifonos freelace de huawei
37464,no encuentro modelo nokia 8 plus
37470,pues es dificil buscar equipos uno a uno marca...
37471,xiaomi mi a3
37472,modem zte


In [76]:
text_data = df.apply(prepare_text,axis = 1)
text_data = text_data.to_frame()
text_data.columns = ["text"]
text_data.head(5)

,text
IDUnico,
37458,"[audifonos, freelace, huawei]"
37464,"[encontrar, modelar, nokia, plus]"
37470,"[pues, dificil, buscar, equipo, marcar, marcar..."
37471,[xiaomi]
37472,"[modem, zte]"


In [77]:
dicc = get_dict(text_data)

In [83]:
max_iter = 5
des_pct = .8
umbral = .6

pct_class,iterations = 0,0
input_data = text_data.copy(deep = True)
o_len = len(input_data)
while pct_class < des_pct and iterations < max_iter:
    classified, input_data = topic_classification(input_data,dicc,umbral)
    if iterations == 0:
        total_class = classified.copy(deep = True)
    else:
        total_class.append(classified)
        
    iterations += 1
    pct_class = len(total_class)/o_len
    print(f"Ronda {iterations}: \n Se han clasificado el {int(pct_class*100)}% de la base")
    
#sound()

Ronda 1: 
 Se han clasificado el 40% de la base
Ronda 2: 
 Se han clasificado el 40% de la base
Ronda 3: 
 Se han clasificado el 40% de la base
Ronda 4: 
 Se han clasificado el 40% de la base
Ronda 5: 
 Se han clasificado el 40% de la base


In [84]:
classified, input_data = topic_classification(text_data,dicc,0.7)
#sound()

In [85]:
classified.sample(n=10)

,text,Value,Topic
IDUnico,,,
43987,"[iphone, telcel]",0.842974,"[iphone, plus, pro, max, gb]"
93221,"[cubo, cable, iphone]",0.862564,"[iphone, plus, pro, max, gb]"
81525,"[buscar, caracteristicas, celular, alpha, cost...",1.199185,"[celular, buscar, pro, motorola, one]"
84978,"[samsung, galaxy]",1.117261,"[samsung, galaxy, notar, plan, redmi]"
75816,"[plan, motorola, moto, plus, moto, z3play]",1.301522,"[motorola, one, plus, plan, moto]"
141839,"[equipo, cotizacion, plan]",0.880442,"[plan, equipar, querer, iphone, equipo]"
82760,"[buscar, informacion, plan, telcel, solo, llev...",2.104400,"[plan, equipar, querer, iphone, equipo]"
125782,"[querer, cotejar, plan, iphone]",1.095259,"[plan, equipar, querer, iphone, equipo]"
38790,"[iphone, plus]",1.083005,"[iphone, plus, pro, max, gb]"


In [86]:
input_data.head(5)

,text
IDUnico,
131072,[ewes]
131077,"[meter, mas, variedad, xiaomi]"
131091,"[hola, gracia, atención, intersa, saber, promo..."
131108,[carnd]
131110,"[plan, obtener, notar, +, 1144, mxp]"


In [87]:
dic2 = get_dict(input_data)
classified2, input_data = topic_classification(input_data,dicc,0.8)
#sound()

NameError: name 'sound' is not defined

In [88]:
classified2.sample(n = 10)

,text,Value,Topic
IDUnico,,,
37957,"[mas, producto, marcar, sony]",0.925713,"[samsung, a50, a70, a80, +]"
138411,"[apple, watch, seriar]",0.959999,"[samsung, a50, a70, a80, +]"
87561,"[paginar, funcionar]",0.934793,"[samsung, a50, a70, a80, +]"
91251,"[moto, one]",0.803904,"[buscar, telefono, celular, telefonos, moto]"
45036,"[alcatel, mas, recentar]",0.849239,"[buscar, telefono, celular, telefonos, moto]"
45867,"[paquete, dato, viajar, extranjero]",0.926242,"[samsung, a50, a70, a80, +]"
142294,[smarphon],0.959999,"[samsung, a50, a70, a80, +]"
83133,"[necesitar, ayudar]",0.886438,"[celular, xiaomi, querer, samsung, aparecer]"
143919,"[pesimo, opción, equipo]",0.925713,"[samsung, a50, a70, a80, +]"
